In [ ]:
# set cuda visible devices
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

import os
if is_notebook():
    os.environ["CUDA_VISIBLE_DEVICES"] = "" #"1"
    # os.environ['CUDA_LAUNCH_BLOCKING']="1"
    # os.environ['TORCH_USE_CUDA_DSA'] = "1"

import matplotlib 
if not is_notebook():
    matplotlib.use('Agg')

# set directory
os.chdir("/nas/ucb/oliveradk/diverse-gen/")

In [ ]:
import json
from functools import partial
from itertools import product
from typing import Optional, Literal, Callable
from tqdm import tqdm
from collections import defaultdict
from dataclasses import dataclass
from pathlib import Path
from copy import deepcopy
from datetime import datetime
from collections import defaultdict

import submitit
from submitit.core.utils import CommandFunction
import nevergrad as ng
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

from losses.loss_types import LossType
from utils.exp_utils import get_executor, get_executor_local, run_experiments
from utils.utils import conf_to_args

In [ ]:
SCRIPT_NAME = "spur_corr_exp.py"
EXP_DIR = Path("output/real_data_exps")
EXP_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
seeds = [1, 2, 3]

# TODO: add aux weights based on tuining
method_configs = {
    "DivDis": {"loss_type": LossType.DIVDIS},
    "TopK 0.1": {"loss_type": LossType.TOPK, "mix_rate_lower_bound": 0.1},
    "TopK 0.5": {"loss_type": LossType.TOPK, "mix_rate_lower_bound": 0.5},
    "DBAT": {"loss_type": LossType.DBAT, "shared_backbone": False, "freeze_heads": True, "binary": True, "batch_size": 16, "target_batch_size": 32},
}

dataset_configs = {
    "waterbirds": {"dataset": "waterbirds", "model": "Resnet50", "epochs": 5, "source_cc": False},
}

configs = {
    (ds_name, method_name, seed): {**ds_config, **method_config, "seed": seed} 
    for (ds_name, ds_config), (method_name, method_config) in product(dataset_configs.items(), method_configs.items())
    for seed in seeds
}

def get_conf_dir(conf_name: tuple):
    ds, method, seed = conf_name
    return f"{ds}_{method}/{seed}"

for (ds_name, method_name), conf in configs.items():
    exp_dir = get_conf_exp_dir(ds_name, method_name, conf["aux_weight"])
    conf["exp_dir"] = exp_dir

# Run Experiments

In [ ]:
executor = get_executor(EXP_DIR, mem_gb=16)
jobs = run_experiments(executor, list(configs.values()), SCRIPT_NAME)